In [ ]:
%load_ext autoreload
%autoreload 2


import json
import os
import matplotlib.pyplot as plt
import numpy as np

from ScheduleDistributionSampler import ScheduleDistributionSampler, activity_map, start_times

In [ ]:
print({a : {} for a in set(activity_map.values())})

In [ ]:
see_activities = list(set(activity_map.values()))
see_activities.remove(None)
individual_histograms = {}

schedule_num = 0
for root, dirs, files in os.walk('data/AMT_Schedules'):
    total_files = len(files) - 4
    fig, ind_plot = plt.subplots(total_files, len(see_activities), sharex=True, sharey=True)
    fig.set_size_inches(80, 50)
    scene_histogram = {}
    for f in files:
        if f.endswith('.json'):
            if f in ['A3EF9IKIU8IAHY.json', 'A3FKOG3VU5MP0X.json','AFER49XXMM4PE.json','A332L3FO6RCY6G.json']:
                continue
            individual_histograms[f[:-5]] = {}
            with open(os.path.join(root,f)) as file:
                sch = json.load(file)
            age = int(sch["questionnaireData"]["age"][:2])
            if sch["questionnaireData"]["gender"] == 'Female': c = [age/60, 0, 0]
            if sch["questionnaireData"]["gender"] == 'Male': c = [0, 0, age/60]
            # ind_plot[schedule_num,0].set_ylabel(str(age) + ', '+gender[0])
            activity_times = {}
            scene_histogram[f] = {k:{s:0 for s in start_times} for k in activity_map.values()}
            for timestring,activities in sch['activitiesChosen'].items():
                start_time = timestring.split('m')[0]
                start_time = int(start_time[:-1]) if start_time[-1] == 'a' else int(start_time[:-1])+12
                if start_time == 24:
                    start_time = 12
                for act in activities:
                    sch_activity = activity_map[act]
                    scene_histogram[f][sch_activity][start_time] = 1
            for iact,act in enumerate(see_activities):
                individual_histograms[f[:-5]][act] = list(scene_histogram[f][act].values())
                ind_plot[schedule_num][iact].bar(scene_histogram[f][act].keys(), scene_histogram[f][act].values())
                ind_plot[schedule_num][iact].set_yticks([])
                ind_plot[schedule_num][iact].set_xticks([])
            ind_plot[schedule_num,0].set_ylabel(f[:-5], rotation=0, labelpad=100)
            schedule_num += 1
    print('Done')
schedule_num -= 1
for iact, act in enumerate(see_activities):
    ind_plot[0,iact].set_title(act.replace('_','\n'))
    ind_plot[schedule_num][iact].set_xticks([6,9,12,15,18,21,24])
    ind_plot[schedule_num][iact].set_xticklabels(['6\nam','','','3\npm','','','12\nam'])

In [ ]:
# with open('data/personaBasedSchedules/individual_histograms.json', 'w') as f:
#     json.dump(individual_histograms, f, indent=4)

In [ ]:
with open('data/personaBasedSchedules/corrected_histograms.json') as f:
    corrected_histograms = json.load(f)

fig, ind_plot = plt.subplots(len(corrected_histograms.keys()), len(see_activities), sharex=True, sharey=True)
fig.set_size_inches(80, 50)

for i,(id,act_hists) in enumerate(corrected_histograms.items()):
    for j,(activity,data) in enumerate(act_hists.items()):
        ind_plot[i][j].bar(start_times, data)
        ind_plot[i][j].set_yticks([])
        ind_plot[i][j].set_xticks([])
        if i==0:
            ind_plot[i][j].set_title(activity.replace('_','\n'))
    ind_plot[i][0].set_ylabel(id, rotation=0, labelpad=100)

In [ ]:
persona_traits = {
'breakfast': {"has":[], "skips":[]}, 
'lunch': {"has":[], "skips":[]}, 
'dinner': {"has":[], "skips":[]}, 
'computer_work': {"less":[], "normal":[], "more":[]}, 
'going_to_the_bathroom': {"uses_bathroom":[], "does_not_use_bathroom":[]}, 
'leave_home': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'come_home': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'playing_music': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'getting_dressed': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'cleaning': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'socializing': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'listening_to_music':  {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'taking_medication': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'take_out_trash': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'kitchen_cleaning': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'wash_dishes': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'brushing_teeth': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'laundry': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'hand_wash_clothes': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'reading': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'showering': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'vaccuum_cleaning': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
'watching_tv': {"morning":[], "evening":[], "multiple_times":[], "not_at_all":[]}, 
}

In [ ]:
cutoff = 10
computer_work_cutoffs = [4, 10]

for i,(id,act_hists) in enumerate(corrected_histograms.items()):
    for j,(activity,data) in enumerate(act_hists.items()):
        np_data = np.array(data)
        if set(list(persona_traits[activity].keys())) == set(["morning", "evening", "multiple_times", "not_at_all"]):
            morning = np_data[:cutoff].sum() > 0
            evening = np_data[cutoff:].sum() > 0
            if morning and evening: persona_traits[activity]["multiple_times"].append(id)
            elif morning: persona_traits[activity]["morning"].append(id)
            elif evening: persona_traits[activity]["evening"].append(id)
            else: persona_traits[activity]["not_at_all"].append(id)
        if set(list(persona_traits[activity].keys())) == set(["has", "skips"]):
            if (np_data.sum() > 0): persona_traits[activity]["has"].append(id)
            else: persona_traits[activity]["skips"].append(id)
        if activity == "computer_work":
            if (np_data.sum() <= computer_work_cutoffs[0]): persona_traits[activity]["less"].append(id)
            elif (np_data.sum() <= computer_work_cutoffs[1]): persona_traits[activity]["normal"].append(id)
            else: persona_traits[activity]["more"].append(id)
        if activity == "going_to_the_bathroom":
            if (np_data.sum() > 0): persona_traits[activity]["uses_bathroom"].append(id)
            else: persona_traits[activity]["does_not_use_bathroom"].append(id)

persona_traits

In [ ]:
with open('data/personaBasedSchedules/persona_traits.json', 'w') as f:
    json.dump(persona_traits, f, indent=4)

persona_trait_frequencies = {act:{trait: len(files) for trait, files in trait_files.items()} for act,trait_files in persona_traits.items()}
persona_trait_frequencies

In [ ]:
# with open('data/personaBasedSchedules/persona_traits.json') as f:
#     persona_traits = json.load(f)

def plot_together(list_of_freqs, ax):
    for i,freqs in enumerate(list_of_freqs):
        freq_exist = np.array(start_times)[np.array(freqs) == 1]
        ax.plot(freq_exist,freq_exist*0 + i, '.')

trait_histograms = {}

for activity in persona_traits:
    trait_histograms[activity] = {}
    fig, axs = plt.subplots(2, len(persona_traits[activity]), sharex=True)
    fig.set_size_inches(20, 5)
    fig.suptitle(activity)
    for i, (trait, files) in enumerate(persona_traits[activity].items()):
        if len(files) == 0:
            continue
        trait_histograms[activity][trait] = list((np.array([np.array(corrected_histograms[f][activity]) for f in files])).mean(0))
        axs[1][i].plot(start_times, trait_histograms[activity][trait], '-.k')
        plot_together([corrected_histograms[f][activity] for f in files], axs[0][i])
        axs[0][i].set_title('{}; p={:.2f}'.format(trait, len(files)/total_files))
    fig.tight_layout()
    plt.savefig('data/personaBasedSchedules/histograms/'+activity+'.jpg')


with open('data/personaBasedSchedules/trait_histograms.json','w') as f:
    json.dump(trait_histograms, f, indent=4)